## Backtesting Data

Currently, I am working with Jeff Sackman's ATP dataset. However, for backtesting I need bookmaker odds, which is not included in this data. I've found another dataset that includes the bookmaker odds. The aim of this dataset is to combine the two to form a dataset that can be used for backtesting.

In [49]:
import os
import pandas as pd

# List of xls years
start_year = 2001
end_year = 2024
files = []

for year in range(start_year, end_year + 1):
    base_path = f"../../data/raw/tennis-data/{year}"
    
    xls_path = base_path + ".xls"
    xlsx_path = base_path + ".xlsx"
    
    if os.path.exists(xls_path):
        files.append(xls_path)
    elif os.path.exists(xlsx_path):
        files.append(xlsx_path)
    else:
        print(f"Warning: no file found for year {year} ({xls_path} or {xlsx_path})")

# Initialize an empty list to store the DataFrames
dfs = []

# Loop through the files, read each one, and append to the list
for file in files:
    df = pd.read_excel(file)
    dfs.append(df)

# Concatenate all DataFrames in the list vertically (stacking rows)
odds_df = pd.concat(dfs, axis=0, ignore_index=True)
sackman_df = pd.read_parquet("../../data/training_data/dataset_v1_full.parquet")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [50]:
sackman_df.columns

Index(['surface', 'draw_size', 'tourney_level', 'tourney_date', 'id_a',
       'name_a', 'hand_a', 'ht_a', 'age_a', 'id_b', 'name_b', 'hand_b', 'ht_b',
       'age_b', 'score', 'best_of', 'round', 'minutes', 'ace_a', 'df_a',
       'svpt_a', '1stIn_a', '1stWon_a', '2ndWon_a', 'SvGms_a', 'bpSaved_a',
       'bpFaced_a', 'ace_b', 'df_b', 'svpt_b', '1stIn_b', '1stWon_b',
       '2ndWon_b', 'SvGms_b', 'bpSaved_b', 'bpFaced_b', 'rank_a',
       'rank_points_a', 'rank_b', 'rank_points_b', 'result', 'p_ace_a',
       'p_ace_b', 'p_df_a', 'p_df_b', 'p_1stIn_a', 'p_1stIn_b', 'p_1stWon_a',
       'p_1stWon_b', 'p_2ndWon_a', 'p_2ndWon_b', 'p_2ndWon_inPlay_a',
       'p_2ndWon_inPlay_b', 'p_bpSaved_a', 'p_bpSaved_b', 'p_rpw_a', 'p_rpw_b',
       'p_retAceAgainst_a', 'p_retAceAgainst_b', 'p_ret1stWon_a',
       'p_ret1stWon_b', 'p_ret2ndWon_a', 'p_ret2ndWon_b',
       'p_ret2ndWon_inPlay_a', 'p_ret2ndWon_inPlay_b', 'p_bpConv_a',
       'p_bpConv_b', 'p_totalPtsWon_a', 'p_totalPtsWon_b', 'dominance_

In [51]:
odds_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'],
      dtype='object')

There are a few rows that can be used as merge points.
1. Winner Name
2. Loser Name
3. Date
4. Surface
5. Best of
6. Set scores

Winner name, loser name and date will be able to match more than 99% of the rows. Duplicate matchings will be removed.

Name old format: Firstname Lastname

Name new format: Lastname FirstInitial.

The old name will be converted to the new format.

Date old format: YYYYMMDD rounded to the first monday

Date new format: YYYY-MM-DD precise day of match

The new date will be rounded to the most recent monday for merging.

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def old_to_new_name(name):
    """
    Convert 'Firstname Lastname' -> 'Lastname F.'
    """
    if pd.isna(name):
        return np.nan
    parts = name.split()
    if len(parts) < 2:
        return name
    first, last = parts[0], " ".join(parts[1:])
    initial = first[0].upper()
    return f"{last} {initial}."

def new_to_old_name(name, old_name_lookup):
    """
    Convert 'Lastname F.' -> approximate 'Firstname Lastname' via lookup table.
    If not found, returns original.
    """
    return old_name_lookup.get(name, name)

def monday_round(date_obj):
    """
    Round a date down to the most recent Monday.
    """
    if isinstance(date_obj, str):
        date_obj = pd.to_datetime(date_obj)
    return date_obj - timedelta(days=date_obj.weekday())

def yyyymmdd_to_monday(yyyymmdd):
    """
    Convert YYYYMMDD integer into datetime, then round to Monday.
    """
    if pd.isna(yyyymmdd):
        return np.nan
    s = str(int(yyyymmdd))
    dt = datetime.strptime(s, "%Y%m%d")
    return monday_round(dt)

def merge_old_new(old_df, new_df):
    df_old = old_df.copy()
    df_new = new_df.copy()

    df_old["name_a_new"] = df_old["name_a"].apply(old_to_new_name)
    df_old["name_b_new"] = df_old["name_b"].apply(old_to_new_name)

    # find winner and loser names
    df_old["old_winner"] = np.where(df_old["result"] == 1,
                                df_old["name_a_new"],
                                df_old["name_b_new"])

    df_old["old_loser"] = np.where(df_old["result"] == 1,
                                df_old["name_b_new"],
                                df_old["name_a_new"])

    # For new_df, Winner/Loser are already in Lastname F. format → keep them
    df_new.rename(columns={"Winner": "winner_new",
                           "Loser": "loser_new"}, inplace=True)

    # old_df tourney_date → Monday
    df_old["merge_date"] = df_old["tourney_date"].apply(yyyymmdd_to_monday)

    # new_df Date (YYYY-MM-DD) → round to Monday
    df_new["merge_date"] = df_new["Date"].apply(lambda d: monday_round(pd.to_datetime(d)))

    df_old["surface_norm"] = df_old["surface"].str.lower().str.strip()
    df_old["best_of_norm"] = df_old["best_of"].astype("Int64")

    df_new["surface_norm"] = df_new["Surface"].str.lower().str.strip()
    df_new["best_of_norm"] = df_new["Best of"].astype("Int64")

    merge_cols_old = ["old_winner", "old_loser", "merge_date",
                      "surface_norm", "best_of_norm"]

    merge_cols_new = ["winner_new", "loser_new", "merge_date",
                      "surface_norm", "best_of_norm"]

    merged = df_old.merge(
        df_new,
        left_on=merge_cols_old,
        right_on=merge_cols_new,
        how="left",
        suffixes=("_old", "_new")
    )

    key_cols = ["id_a", "id_b", "merge_date"]
    dup_mask = merged.duplicated(subset=key_cols, keep=False)

    # keep non duplicated rows
    merged = merged[~dup_mask].copy()

    # remove rows with no odds
    odds_cols = ["PSW", "PSL", "B365W", "B365L", "IWW", "IWL"]
    merged = merged.dropna(subset=odds_cols, how="all")

    return merged

In [ ]:
merged_df = merge_old_new(sackman_df, odds_df)

og_rows = odds_df.shape[0]
merged_rows = merged_df.shape[0]

print("rows in sackman_df", og_rows)
print("rows in merged_df", merged_rows)
print("percent lost", (og_rows - merged_rows) / og_rows)

rows in sackman_df 63739
rows in merged_df 44136
percent lost 0.30755110685765386


0